In [1]:
import numpy as np
import pandas as pd
import boto3

In [2]:
# Let's use Amazon S3
# The Access Key ID and Secret Access Key are in ~/.aws/credentials
# Region is set in ~/.aws/config
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

athena-query-bucket-testda24
aws-glue-assets-936480984004-eu-north-1
hacksprint-2023
temp-transformation-rs
vehicle-bucket-alarm


In [4]:
# Print the file names in the bucket
revolvos_bucket = s3.Bucket('hacksprint-2023')
for object in revolvos_bucket.objects.all():
    print(object.key)

Telematics Data Dictionary 2023_02_14.xlsx
Vehicle_Fault.csv
Vehicle_Fault_Alarm.csv
Vehicle_Fault_Alarm_Item.csv
Vehicle_Location.csv
Vehicle_Specs.csv


In [5]:
%%time
# Load one of the files into a dataframe
obj = s3.Object('hacksprint-2023', 'Vehicle_Fault_Alarm.csv').get()
df = pd.read_csv(obj['Body'], low_memory=False)

CPU times: user 30.4 s, sys: 5.39 s, total: 35.8 s
Wall time: 1min 10s


In [7]:
print(df.dtypes)

FAULT_ALARM_ID                 int64
COMPANY_ID                     int64
VEHICLE_ID                     int64
MESSAGE_DATE_TIME             object
VEHICLE_MESSAGE_ID             int64
LONGITUDE                    float64
LATITUDE                     float64
CITY_NAME                     object
STATE_CD                      object
FAULT_COUNT                    int64
COMPONENT_TYPE                object
FMI_ID                         int64
FIRST_OCCURRENCE              object
LAST_OCCURRENCE               object
ECU_TYPE                      object
ECU_ID                         int64
COMPONENT_ID                   int64
FAULT_ACTIVE                  object
MSW                          float64
DST1                         float64
DST2                         float64
EDR_REPORT_IDENTIFICATION    float64
EDR_VFA_LINK                 float64
FAULT_FILTER_ID              float64
MALFUNCTION_LAMP_STATUS       object
RED_LAMP_STATUS               object
YELLOW_LAMP_STATUS            object
P

In [8]:
print(df.head())

   FAULT_ALARM_ID  COMPANY_ID  VEHICLE_ID    MESSAGE_DATE_TIME  \
0      2343110281      649030   100212831  7/1/2022 2:46:58 AM   
1      2343111175      649030   100212831  7/1/2022 2:47:04 AM   
2      2343124523      649030   100212831  7/1/2022 3:03:02 AM   
3      2343125320      649030   100212831  7/1/2022 3:03:07 AM   
4      2345440161      649030   100212831  7/2/2022 4:13:19 PM   

   VEHICLE_MESSAGE_ID   LONGITUDE   LATITUDE CITY_NAME STATE_CD  FAULT_COUNT  \
0         10311654432 -118.983314  54.647507       NaN      NaN          103   
1         10311657676 -118.983314  54.647507       NaN      NaN          103   
2         10311704845 -118.983314  54.647507       NaN      NaN          104   
3         10311707803 -118.983314  54.647507       NaN      NaN          104   
4         10319802899 -119.551490  55.370797     Hythe       AB          105   

   ... EDR_VFA_LINK  FAULT_FILTER_ID MALFUNCTION_LAMP_STATUS RED_LAMP_STATUS  \
0  ...          NaN           24.022      

Insert magical cleaning...

In [ ]:
%%time
# Send the cleaned dataframe content back to a new CSV file on the S3 bucket
csv_clean = df.to_csv(index=False)
_ = s3.Object('hacksprint-2023', 'Vehicle_Fault_Alarm-clean.csv').put(Body=csv_clean)